In [ ]:
import numpy as np
import gurobipy as gp
from algorithms import *

Variables to be used to generate nodes and cluster

In [ ]:
np.random.seed(42)

# Number of nodes
n = 10000
# Number of clusters
p = 20

capacity_mean = 10
capacity_stddev = 2
weight_mean = 1
weight_stddev = 0.1
lambda_param = 0.5

Generate random nodes, capacities, and weights

In [ ]:
np.random.seed(42)

# Generate 2D nodes and random capacities for clusters
nodes, capacities = generate_instances(n, p, capacity_mean, capacity_stddev)
# Generate random weight for each node
weights = generate_weights(n, weight_mean, weight_stddev)

Run CCP - Gurobi clustering algorithm

In [ ]:
ccp_clusters, ccp_time, obj_val = solve_model(nodes, capacities, weights, lambda_param,'ccp')
print("Elapsed time:",ccp_time,'s')
plot_clusters(ccp_clusters,nodes,'CCP')

Calculate In Sample Stability for CCP - Gurobi

In [ ]:
insample_stability_score = insample_stability(ccp_clusters, nodes, capacities, weights, lambda_param, num_runs=10)
print("Insample Stability:",insample_stability_score)

Calculate Out Sample Stability for CCP - Gurobi

In [ ]:
outsample_stability_score = outsample_stability(n, p, weight_mean, weight_stddev, capacity_mean, capacity_stddev, lambda_param, num_runs=10)

print("Outsample Stability:",outsample_stability_score)

Calculate VSS for CCP - Gurobi

In [ ]:
num_simulations = 100
np.random.seed(42)
VSS = calculate_VSS(n, p, capacity_mean, capacity_stddev, weight_mean, weight_stddev, lambda_param, num_simulations)
print(f"The Value of Stochastic Solution (VSS) is {VSS:.2f}")

Run K-Means with Capacity Constraints (Heuristic) clustering algorithm

In [ ]:
kmcc_clusters, kmcc_time = kmcc_clustering(nodes, capacities, weights)
print("Elapsed time:",kmcc_time,'s')
plot_clusters(kmcc_clusters,nodes,'KMCC')

In [ ]:
kmcc_insample_stability_score = KMCC_insample_stability(kmcc_clusters, nodes, capacities, weights, num_runs=3)
print("KMCC Insample Stability:",kmcc_insample_stability_score)

In [ ]:
kmcc_outsample_stability_score = KMCC_outsample_stability(n, p, weight_mean, weight_stddev, capacity_mean, capacity_stddev, num_runs=2)

print("KMCC Outsample Stability:",kmcc_outsample_stability_score)